In [440]:
# Initialize Otter
import otter
grader = otter.Notebook("ps4.ipynb")

### Question 1
In this question you will implement a class called `Matrix`. A matrix will be modeled as a collection of vectors all of the same dimension, and you may use the `Vector` class that we created in Lecture 5 (included with this problem set as `vector.py`) to implement it:

In [441]:
from vector import Vector
class Matrix:
    
    def __init__(self, *args):
        if not all(isinstance(col, Vector) for col in args):
            raise TypeError('Inputs have to be vectors.')
        dims = {len(v) for v in args} #set comprehension, how many different are there? if more than 1 value, then there are more vecs with different lenght
        if len(dims)>1 or len(dims)==0:
            raise ValueError('Vectors have to have same dimension.')
        
        row_to_col = [x for x in zip(*args)]
        self._data = [Vector(*x) for x in row_to_col]
        
    def __str__(self):
        return '\n'.join([str(x) for x in self._data])
    
    @property
    def dims(self):
        return (len(self._data),len(self._data[0]))
    
    def __iter__(self):
        col_to_row = zip(*self._data)
        return iter([Vector(*x) for x in col_to_row])
    
    def __add__(self, other):
        if not self._conformable(other):
            raise ValueError("Matrices must be the same size.")
        return Matrix(*[a + b for a, b in zip(self, other)])
            
    def _conformable(self, other):
        return isinstance(other, Matrix) and self.dims == other.dims
    
    def __mul__(self, other):
        import numbers
        if not isinstance(other, numbers.Number):
            return NotImplemented
        return Matrix(*[a * b for a, b in zip(self, Vector(*tuple([other]*len(self._data[0]))))])
    
    def __rmul__(self, other):
        return self.__mul__(other)
    
    def __getitem__(self, i):
        return self._data[i[0]][i[1]]
    
    def transpose(self):
        return Matrix(*[a for a in self._data])
    
    def __matmul__(self, other):
        if not self.dims == (other.dims[1],other.dims[0]):
            raise ValueError('Matrices cannot be multiplied - wrong dimensions')
        
        cols = [Vector(*v) for v in zip(*self)]
        res = []
        r = []
        for row in self._data:
            for col in cols:
                r.append(Vector.dot(row, col))
                if len(r) > len(self._data)-1:
                    res.append(r)
                    r=[]
        return Matrix(*[Vector(*x) for x in res])
    ...

**1(a)** (1 pt) The constructor of `Matrix` should accept one or more `Vector` objects, which will represent the *columns* of the matrix.

```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(5, 6, 7)
>>> v3 = Vector(8, 9, 10)
>>> Matrix(v1)
<__main__.Matrix at 0x10efbabb0>
>>> Matrix(v1, v2)
<__main__.Matrix at 0x117023640>
>>> Matrix(v1, v2, v3)
<__main__.Matrix at 0x117023040>
```

Additionally, it should perform the following checks:

1.  It should verify that each argument is of the correct type:
    ```
    >>> Matrix(1, "", v1)
    TypeError: Input arguments should be vectors
    ```
    
2.  It should verify that each argument has the same dimension.

    ```
    >>> v1 = Vector(1, 2, 3)
    >>> v2 = Vector(5, 6)
    >>> Matrix(v1, v2)
    ValueError: Vectors should all have the same dimension
    ```

In [442]:
grader.check("1a")

1a results: All test cases passed!

**1(b)** (2 pts) Calling `str` on a `Matrix` object should produce a nice looking text representation:

```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> v3 = Vector(7, 8, 9)
>>> M = Matrix(v1, v2, v3)
>>> print(M)
(1, 4, 7)
(2, 5, 8)
(3, 6, 9)
```

In [443]:
grader.check("q1b")

q1b results: All test cases passed!

**1(c)** (1 pt.) A matrix should have a attribute `dims` which returns a tuple of its dimensions (number of rows and columns):
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> M.dims
(3, 2)
```

In [444]:
grader.check("q1c")

q1c results: All test cases passed!

**1(d)** (1 pt.) A matrix should be iterable. Iterating over the matrix should return each of its columns as vectors.
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> for col in M: print(col)
(1, 2, 3)
(4, 5, 6)
```

In [445]:
grader.check("q1d")

q1d results: All test cases passed!

**1(e)** (2 pts) Two matrices of the same dimensions can be added together. 
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> print(M + M)
(2, 8)
(4, 10)
(6, 12)
```
However, matrices of different dimensions cannot be added together:
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> Matrix(v1) + Matrix(v1, v2)
ValueError: Cannot add matrices of different dimensions
```

In [446]:
grader.check("q1e")

q1e results: All test cases passed!

**1(f)** (2 pts) Matrices can be multipled by a numerical constant:
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> print(M * 2.5)
(2.5, 10.0)
(5.0, 12.5)
(7.5, 15.0)
>>> print("a" * M)
TypeError: ...
```

In [447]:
grader.check("q1f")

q1f results: All test cases passed!

**1(g)** (2 pts) Individual entries of a matrix can be accessed using the notation `M[i,j]`:
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> print(M)
>>> M[0, 0]
1
>>> M[2, 1]
6
>>> M[3, 3]
IndexError: ...
```

In [448]:
grader.check("q1g")

q1g results: All test cases passed!

**1(h)** (3 pts) A matrix can be transposed:
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> print(M)
(1, 4)
(2, 5)
(3, 6)
>>> print(M.transpose())
(1, 2, 3)
(4, 5, 6)
```

In [449]:
grader.check("q1h")

q1h results: All test cases passed!

**1(i)** (2 pts) Two matrices of conformable dimensions can be multiplied using the matrix multiplication operator (`@`):
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> print(M @ M.transpose())
(17, 22, 27)
(22, 29, 36)
(27, 36, 45)
>>> M @ M
ValueError: Incompatible matrix dimensions
```

In [450]:
grader.check("q1i")

q1i results: All test cases passed!

**1(j)** (2 pts) Matrices can be tested for equality:
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> M == M
True
>>> M == Matrix(v1)
False
>>> M == 1.0
False
>>> M == "matrix"
False
```

In [451]:
grader.check("q1j")

q1j results: All test cases passed!

### Question 2
[Connect Four](https://en.wikipedia.org/wiki/Connect_Four) is a two-player game where players drop colored discs vertically into a $6\times 7$ grid, with the object of being the first to form a horizontal, vertical, or diagonal line of four of one's own discs. If you've never played before and want to understand how the game works, [here is a free online version](https://www.cbc.ca/kids/games/play/connect-4).

In this question, you will implement Connect Four as a Python class. Your implementation will be slightly more general: it will allow the user to specify the numbers of rows and columns of the grid, as well as the number $k$ of markers in a row that you need to win. So, traditional Connect Four is `ConnectK(rows=6, cols=7, k=4)` in the class shown below.

In [452]:
class ConnectK:
    '''
    The game Connect K.
    
    Args:
        rows, cols: the dimensions of the grid. defaults to (6, 7) if not specified.
        k: the number of markers in a row a player needs in order to win, 
           defaults to 4 if not specified.
        
    Raises:
        ValueError if rows, cols and k are not all positive integers.
    '''
    
    def __init__(self, rows=6, cols=7, k=4):
        self._rows = rows
        self._cols = cols
        self._k = k
        
        if not (isinstance(self._rows,int) and self._rows > 0 and isinstance(self._cols,int) and self._cols > 0 and isinstance(self._k,int) and self._k > 0):
            raise ValueError('Input parameters must be positive integers.')

        self._board = [['-']*self._rows for x in range(self._cols)]
        self._round = 0
        ...
        
    @property
    def rows(self):
        'the number of rows in the grid'
        return self._rows
        ...

    @property
    def cols(self):
        'the number of columns in the grid'
        return self._cols
        ...
        
    @property
    def k(self):
        'the number of markers in a row needed to win'
        return self._k
        ...
        
    @property
    def possible_moves(self):
        'a list of the currently possible moves (open columns)'
        
        possible_moves = []
        
        for col_i in range(self._cols):
            if '-' in self._board[col_i]:
                possible_moves.append(col_i)
                
        return possible_moves
        ...
    
    
    def __repr__(self):
        rep = 'ConnectK(rows=' + str(self._rows) + ', ' + 'cols=' + str(self._cols) + ', ' +  'k=' + str(self._k) + ')'
        return rep
        ...
        
    def __str__(self):
        'Return a string representation of the board'
        cols_to_lst = list(map(list, zip(*self._board)))
        return '\n'.join([str(x).strip('[]').replace(',','').replace(' ','').replace("'",'') for x in cols_to_lst])
        ...
    
    
    def move(self, i):
        '''
        The player whose turn it is drops their disc in column i.
        
        Args:
            i: The 0-indexed column into which the player drop their disc.
            
        Returns:
            If the move results in the player winning the game, this function returns 0/1 representing the winner.
            Otherwise, it returns None.
            
        Raises:
            - ValueError if the move is not valid (column is full, or a non-existent column was specified).
        '''
        board = self._board
        round_ = self._round
        
        # 1 plays on rounds that are even
        
        try:
            last_empty_row = max(l for l, v in enumerate(board[i]) if v == '-')
            self._round += 1
        except:
            raise ValueError('Column is either full or invalid.')
            
        if not round_ % 2 == 0 and round_ != 0:  # Player 1 Turn
            board[i][last_empty_row] = 1
        else:
            board[i][last_empty_row] = 0         # Player 0 Turn
    
        ### COLS CHECK FOR WIN ###
        c_cons_zeros = []
        c_cons_ones = []     
        
        # append consecutive numbers in a row
        for a in range(1,self._rows):
            if self._rows > 1:
                if (board[i][a] == board[i][a-1] == 0):
                    c_cons_zeros.append(0)

                if (board[i][a] == board[i][a-1] == 1):
                    c_cons_ones.append(1)
            
        ### ROWS CHECK FOR WIN ###
        r_cons_zeros = []
        r_cons_ones = []  
        
        # Swap cols and rows for easier list operations
        cols_to_rows = list(map(list, zip(*board)))
        # append consecutive numbers in a row
        for a in range(1,self._cols):
            if (cols_to_rows[last_empty_row][a] == cols_to_rows[last_empty_row][a-1] == 0):
                r_cons_zeros.append(0)
        
            if (cols_to_rows[last_empty_row][a] == cols_to_rows[last_empty_row][a-1] == 1):
                r_cons_ones.append(1)
 
        ### DIAG CHECK FOR WIN ###
        d_cons_zeros = []
        d_cons_ones = []
        
        max_col = self._cols
        max_row = self._rows
        f_diag = [[] for x in range(max_row + max_col - 1)]
        b_diag = [[] for x in range(len(f_diag))]
        min_bdiag = -max_row + 1
        
        # Get all diags
        diags = []

        for x in range(max_col):
            for y in range(max_row):
                f_diag[x+y].append(cols_to_rows[y][x])
                b_diag[x-y-min_bdiag].append(cols_to_rows[y][x])

        for f, b in zip(f_diag,b_diag):
            if len(f) >= self._k:
                diags.append(f)
            if len(b) >= self._k:
                diags.append(b)
         
        # append consecutive numbers in a row
        if len(diags) > 0:
            for diag in diags:
                for a in range(len(diag)):
                    if (diag[a] == diag[a-1] == 0):
                        d_cons_zeros.append(0)

                    if (diag[a] == diag[a-1] == 1):
                        d_cons_ones.append(1)

        # check for victory, return victor or None
        if len(r_cons_zeros) >= self._k-1 or len(c_cons_zeros) >= self._k-1 or len(d_cons_zeros) >= self._k-1:
            return 0
        
        elif len(r_cons_ones) >= self._k-1 or len(c_cons_ones) >= self._k-1 or len(d_cons_ones) >= self._k-1:
            return 1   
        
        else:
            return None
        ...

**2(a)** (2 pts) The constructor should accept three arguments, `rows`, `columns`, and `k`, and validate them:
```
>>> ConnectK(1, 2, 3)
ConnectK(rows=1, cols=2, k=3)
>>> ConnectK(0, 2, 3)
ValueError: invalid rows, cols and/or k
```
The default arguments for rows, columns, and k should be 6, 7 and 4, respectively:
```
>>> ConnectK()
ConnectK(rows=6, cols=7, k=4)
```
The values of each attribute should be accessible as properties:
```
>>> c = ConnectK(5, 6, 7)
>>> c.rows
5
>>> c.cols
6
>>> c.k
7
```

In [453]:
grader.check("q2a")

q2a results: All test cases passed!

**2(b)** (3 pts) Converting a `ConnectK` game to a string should return a text representation of the grid. The first player is labeled `0`, the second player is labeled `1`, and empty grid squares are shown as a dash `-`.
```
>>> c = ConnectK()
>>> print(c)
-------
-------
-------
-------
-------
-------
>>> c.move(0)
>>> c.move(1)
>>> c.move(0)
>>> print(c)
-------
-------
-------
-------
0------
01-----
```


In [454]:
grader.check("q2b")

q2b results: All test cases passed!

**2(c)** (8 pts) The `move` method should accept a single argument, `i`, and simulate the player whose turn it is dropping their marker in column `i`. If the move results in the player winning the game, the method returns `0` if the first player won, and `1` if the second player won. Otherwise it returns nothing. If an invalid column is passed in, the method should raise a `ValueError`.
```
>>> c = ConnectK(1, 3, 2)
ConnectK(rows=1, cols=3, k=2)
>>> c.move(0) 
>>> c.move(2)
>>> print(c)
0-1
>>> c.move(0)  # column zero is full
ValueError: ...
>>> c.move("A")  # invalid column
ValueError: ...
>>> c.move(1)  # player 0 wins
0
>>> print(c)
001
```

In [455]:
grader.check("q2c")

q2c results: All test cases passed!

**2(d)** (2 pts) The `possible_moves` property should return a list of all the possible moves for the current player. 
```
>>> c = ConnectK(1, 3, 2)
ConnectK(rows=1, cols=3, k=2)
>>> c.possible_moves
[0, 1, 2]
>>> c.move(1)
>>> print(c)
-0-
>>> c.possible_moves
[0, 2]
>>> c.move(2)
>>> print(c)
-01
>>> c.possible_moves
[0]
```

In [456]:
grader.check("q2d")

q2d results: All test cases passed!

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [457]:
grader.check_all()

1a results: All test cases passed!

q1b results: All test cases passed!

q1c results: All test cases passed!

q1d results: All test cases passed!

q1e results: All test cases passed!

q1f results: All test cases passed!

q1g results: All test cases passed!

q1h results: All test cases passed!

q1i results: All test cases passed!

q1j results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q2c results: All test cases passed!

q2d results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Upload this .zip file to Gradescope for grading.

In [458]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)